In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/flight-delays/flights.csv
/kaggle/input/flight-delays/airports.csv
/kaggle/input/flight-delays/airlines.csv


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import datetime

import seaborn as sns



In [3]:
import category_encoders as ce   

In [48]:
from sklearn.pipeline import Pipeline

In [4]:
flight=pd.read_csv("/kaggle/input/flight-delays/flights.csv",low_memory=False)

In [6]:
flight_w=pd.read_csv("/kaggle/input/flight-delays/flights.csv",low_memory=False)

In [5]:
missing_flight = flight.isnull().sum(axis=0).reset_index()
missing_flight.columns = ['variable', 'missing values']
missing_flight['filling factor (%)']=(flight.shape[0]-missing_flight['missing values'])/flight.shape[0]*100
missing_flight.sort_values('filling factor (%)').reset_index(drop = True)
ar=missing_flight[missing_flight['filling factor (%)']<20]['variable']
drop_l=[]
for i in range(len(ar)):
    drop_l.append(ar.values[i])
    
flight.drop(drop_l,axis=1,inplace=True)
flight.dropna(inplace=True)
flight['Date']=pd.to_datetime(flight[['YEAR','MONTH','DAY']])
flight.drop(['DIVERTED','CANCELLED'],axis=1,inplace=True)


In [7]:
flight.drop(['FLIGHT_NUMBER','TAIL_NUMBER','SCHEDULED_DEPARTURE','DEPARTURE_TIME','SCHEDULED_ARRIVAL','ARRIVAL_TIME','TAXI_OUT',
 'WHEELS_OFF',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'AIR_TIME','WHEELS_ON',
 'TAXI_IN',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME'],axis=1,inplace=True)

flight.drop('YEAR',axis=1,inplace=True)





In [8]:
flight[['AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT']]=flight[['AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT']].astype('category')

In [ ]:
flight.dtypes

In [ ]:
flight.head()

In [9]:
flight['AIRLINE'].value_counts()

WN    1242403
DL     870275
AA     712935
OO     576814
EV     554752
UA     507762
MQ     278791
B6     262042
US     194223
AS     171439
NK     115193
F9      90090
HA      76041
VX      61248
Name: AIRLINE, dtype: int64

In [11]:
flight_wn=flight[flight['AIRLINE']=='WN']

In [39]:
flight_wn.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,DISTANCE,ARRIVAL_DELAY,Date
155,1,1,4,WN,SFO,DEN,-2.0,967,-2.0,2015-01-01
184,1,1,4,WN,BOS,BWI,-4.0,369,-26.0,2015-01-01
185,1,1,4,WN,LAS,BWI,2.0,2106,-25.0,2015-01-01
186,1,1,4,WN,ABQ,HOU,7.0,759,22.0,2015-01-01
187,1,1,4,WN,PHX,SLC,1.0,507,-6.0,2015-01-01


In [30]:
# flight_w[flight_w[flight_w.iloc[:,30].astype(bool)].notna()]

In [33]:
flight_w.loc[lambda x : x>0]

TypeError: '>' not supported between instances of 'str' and 'int'

In [35]:
flight_wnon=flight_w[flight_w.iloc[:,30].notna()]

In [38]:
flight_wnon[flight_wnon.iloc[:,30].astype(bool)]

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
70,2015,1,1,4,AA,1057,N3ASAA,DFW,MIA,515,...,1038.0,102.0,0,0,NaN,0.0,0.0,0.0,0.0,102.0
106,2015,1,1,4,DL,824,N3736C,PHX,DTW,540,...,1331.0,128.0,0,0,NaN,0.0,0.0,0.0,0.0,128.0
165,2015,1,1,4,EV,5976,N27200,BUF,EWR,555,...,1101.0,226.0,0,0,NaN,13.0,0.0,0.0,0.0,213.0
171,2015,1,1,4,OO,5547,N910SW,ONT,DEN,559,...,1116.0,116.0,0,0,NaN,0.0,0.0,0.0,0.0,116.0
174,2015,1,1,4,UA,247,N462UA,PHX,IAH,600,...,1131.0,123.0,0,0,NaN,12.0,0.0,0.0,0.0,111.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5812702,2015,12,31,4,VX,879,N838VA,DAL,LAX,1245,...,1728.0,203.0,0,0,NaN,0.0,0.0,0.0,193.0,10.0
5814141,2015,12,31,4,AA,895,N813AW,DSM,PHX,1425,...,1721.0,48.0,0,0,NaN,8.0,0.0,0.0,32.0,8.0
5814330,2015,12,31,4,MQ,3161,N655MQ,GRR,DFW,1438,...,1724.0,44.0,0,0,NaN,30.0,0.0,0.0,12.0,2.0
5816038,2015,12,31,4,WN,5292,N245WN,RNO,SAN,1645,...,1854.0,44.0,0,0,NaN,1.0,0.0,0.0,36.0,7.0


In [43]:
flight_wnon[flight_wnon.iloc[:,30].astype(bool)].to_csv("flight_weather.csv",index=False)

In [44]:
flight_wn.to_csv("flight_wn.csv",index=False)

In [55]:
f_w=flight_wnon[flight_wnon.iloc[:,30].astype(bool)]

In [56]:
encoding_pipeline = Pipeline([
  ('encode_airline', ce.HashingEncoder(n_components=15, cols=['AIRLINE'],return_df=True)),
  ('encode_origin', ce.TargetEncoder(cols=['ORIGIN_AIRPORT'], return_df=True)),
  ('encode_destination', ce.TargetEncoder(cols=['DESTINATION_AIRPORT'], return_df=True)),
  ('encode_month', ce.TargetEncoder(cols=['MONTH'], return_df=True))
])

# Get the encoded dataset:
df_encoded = encoding_pipeline.fit_transform(f_w, f_w['ARRIVAL_DELAY'])

# , f_w['ARRIVAL_DELAY']

In [59]:
encoding_pipeline = Pipeline([
  
  ('encode_origin', ce.TargetEncoder(cols=['ORIGIN_AIRPORT'], return_df=True)),
  ('encode_destination', ce.TargetEncoder(cols=['DESTINATION_AIRPORT'], return_df=True)),
  ('encode_month', ce.TargetEncoder(cols=['MONTH'], return_df=True))
])

# Get the encoded dataset:
encoded_wn = encoding_pipeline.fit_transform(flight_wn, flight_wn['ARRIVAL_DELAY'])


In [60]:
encoded_wn.to_csv("encoded_flight_wn.csv",index=False)

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,DISTANCE,ARRIVAL_DELAY,Date
155,3.389466,1,4,WN,5.973177,1.657896,-2.0,967,-2.0,2015-01-01
184,3.389466,1,4,WN,3.150531,2.987091,-4.0,369,-26.0,2015-01-01
185,3.389466,1,4,WN,6.841868,2.987091,2.0,2106,-25.0,2015-01-01
186,3.389466,1,4,WN,6.227839,4.195129,7.0,759,22.0,2015-01-01
187,3.389466,1,4,WN,6.450217,6.956344,1.0,507,-6.0,2015-01-01
...,...,...,...,...,...,...,...,...,...,...
5818908,6.720893,31,4,WN,4.289236,7.956295,6.0,761,-7.0,2015-12-31
5818909,6.720893,31,4,WN,4.289236,9.199231,-1.0,481,0.0,2015-12-31
5818927,6.720893,31,4,WN,4.289236,7.196217,-5.0,528,-13.0,2015-12-31
5818928,6.720893,31,4,WN,4.289236,8.161509,-2.0,666,-25.0,2015-12-31


In [58]:
df_encoded.to_csv("encoded_flight_weather.csv",index=False)